In [1]:
# andas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
import requests
import datetime
import json


# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [2]:
#connect to API and get a content

url = "https://yh-finance.p.rapidapi.com/stock/v2/get-balance-sheet"

querystring = {"symbol":"MFGP","region":"US"}

headers = {
	"X-RapidAPI-Key": "ffb9286af0msha9e497d853b5065p1c1fecjsn7f116fe31a99",
	"X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.content)

b'{"financialsTemplate":{"code":"N","maxAge":1},"cashflowStatementHistory":{"cashflowStatements":[{"changeToLiabilities":{"raw":53700000,"fmt":"53.7M","longFmt":"53,700,000"},"totalCashflowsFromInvestingActivities":{"raw":-75900000,"fmt":"-75.9M","longFmt":"-75,900,000"},"netBorrowings":{"raw":-193600000,"fmt":"-193.6M","longFmt":"-193,600,000"},"totalCashFromFinancingActivities":{"raw":-301500000,"fmt":"-301.5M","longFmt":"-301,500,000"},"changeToOperatingActivities":{"raw":14100000,"fmt":"14.1M","longFmt":"14,100,000"},"issuanceOfStock":{"raw":400000,"fmt":"400k","longFmt":"400,000"},"netIncome":{"raw":-424400000,"fmt":"-424.4M","longFmt":"-424,400,000"},"changeInCash":{"raw":-178800000,"fmt":"-178.8M","longFmt":"-178,800,000"},"endDate":{"raw":1635638400,"fmt":"2021-10-31"},"repurchaseOfStock":{"raw":-27200000,"fmt":"-27.2M","longFmt":"-27,200,000"},"effectOfExchangeRate":{"raw":-2000000,"fmt":"-2M","longFmt":"-2,000,000"},"totalCashFromOperatingActivities":{"raw":200600000,"fmt":"2

In [3]:
# create DataFrame
data=pd.json_normalize(response.json())

In [4]:
# DataFrame Visual Check
data.head()

symbol  loading errorList financialsTemplate.code  \
0   MFGP    False        []                       N   

   financialsTemplate.maxAge  \
0                          1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [5]:
# Yearly or Quarterly Data - depends from Column
data_cash=data['cashflowStatementHistory.cashflowStatements']

In [6]:
# Check number of lists
print("Number of sub-tables:", len(data_cash[0]))
print("Number of columns in 1st sub-table:", len(data_cash[0][0]))
print("Number of columns in 2nd sub-table:", len(data_cash[0][1]))
print("Number of columns in 3rd sub-table:", len(data_cash[0][2]))
print("Number of columns in 4th sub-table:", len(data_cash[0][3]))

Number of sub-tables: 4
Number of columns in 1st sub-table: 19
Number of columns in 2nd sub-table: 21
Number of columns in 3rd sub-table: 21
Number of columns in 4th sub-table: 21


In [7]:
# Create a List of available columns
sectors=[]
len(data_cash[0][2])
for z in data_cash[0][2]:
    sectors.append(z)
# Remove unnessesary columns
sectors.remove('maxAge')
print(sectors)

['changeToLiabilities', 'totalCashflowsFromInvestingActivities', 'netBorrowings', 'totalCashFromFinancingActivities', 'changeToOperatingActivities', 'issuanceOfStock', 'netIncome', 'changeInCash', 'endDate', 'repurchaseOfStock', 'effectOfExchangeRate', 'totalCashFromOperatingActivities', 'depreciation', 'otherCashflowsFromInvestingActivities', 'dividendsPaid', 'changeToInventory', 'changeToAccountReceivables', 'otherCashflowsFromFinancingActivities', 'changeToNetincome', 'capitalExpenditures']


In [8]:
# Create list for each column
investments=[]
changeToLiabilities=[]
totalCashflowsFromInvestingActivities=[]
netBorrowings=[]
totalCashFromFinancingActivities=[]
changeToOperatingActivities=[]
issuanceOfStock=[]
netIncome=[]
changeInCash=[]
endDate=[]
repurchaseOfStock=[]
totalCashFromOperatingActivities=[]
depreciation=[]
changeToInventory=[]
changeToAccountReceivables=[]
otherCashflowsFromFinancingActivities=[]
changeToNetincome=[]
capitalExpenditures=[]
dividendsPaid=[]
otherCashflowsFromInvestingActivities=[]
effectOfExchangeRate=[]

In [9]:
# Parse data and add it to lists

#Function with name of Column
def collect(column):
     #Create local variable from the column's name
    b=globals()[column]
     #Marker for NaN value   
    NA_filler=0 
    #Number of sub-tables
    for list_number in range(4):
        #Pick particular column from sub-table
        for zz in data_cash.values[0][list_number]:
            #Take value from the sub-table & column
            if zz==column:
                if "fmt" not in data_cash.values[0][list_number][column]:xx="NaN"
                else: xx=data_cash.values[0][list_number][column]['fmt']
                #If value exist it disable NaN value checker
                NA_filler=1
                #Remove "k" letters and convert number in Mio and add to list
                if xx == None:
                    xx="NaN"
                    b.append(xx)
                elif 'k' in xx:
                    ff=xx.strip('k')
                    ff=float(ff)/1000
                    b.append(ff)
                #Remove "M" letter and add to list
                elif  'M' in xx:
                    tt=xx.strip('M')
                    b.append(tt)
                elif  'B' in xx:
                    mm=xx.strip('B')
                    mm=float(mm)*1000
                    b.append(mm)
                #Add to list if all conditions full-field
                else: 
                    b.append(xx)
        #Add NaN value if value is not exsist 
        if NA_filler==0:
            b.append("NaN")

In [10]:
# Launch the process
for column in sectors:
    collect(column)

In [11]:
balance_sheet=data['balanceSheetHistory.balanceSheetStatements']

In [12]:
print("Number of sub-tables:", len(balance_sheet[0]))
print("Number of columns in 1st sub-table:", len(balance_sheet[0][0]))
print("Number of columns in 2nd sub-table:", len(balance_sheet[0][1]))
print("Number of columns in 3rd sub-table:", len(balance_sheet[0][2]))
print("Number of columns in 4th sub-table:", len(balance_sheet[0][3]))

Number of sub-tables: 4
Number of columns in 1st sub-table: 27
Number of columns in 2nd sub-table: 26
Number of columns in 3rd sub-table: 27
Number of columns in 4th sub-table: 28


In [13]:
sectors_b=[]

for z in balance_sheet[0][3]:
    sectors_b.append(z)
sectors_b.remove('maxAge')
print(sectors_b)


['intangibleAssets', 'capitalSurplus', 'totalLiab', 'totalStockholderEquity', 'minorityInterest', 'otherCurrentLiab', 'totalAssets', 'endDate', 'commonStock', 'otherCurrentAssets', 'retainedEarnings', 'otherLiab', 'goodWill', 'treasuryStock', 'otherAssets', 'cash', 'totalCurrentLiabilities', 'shortLongTermDebt', 'otherStockholderEquity', 'propertyPlantEquipment', 'totalCurrentAssets', 'longTermInvestments', 'netTangibleAssets', 'netReceivables', 'longTermDebt', 'inventory', 'accountsPayable']


In [14]:
intangibleAssets=[]
capitalSurplus=[]
totalLiab=[]
totalStockholderEquity=[]
minorityInterest=[]
deferredLongTermLiab=[]
otherCurrentLiab=[]
totalAssets=[]
commonStock=[]
otherCurrentAssets=[]
retainedEarnings=[]
otherLiab=[]
goodWill=[]
treasuryStock=[]
otherAssets=[]
cash=[]
totalCurrentLiabilities=[]
deferredLongTermAssetCharges=[]
shortLongTermDebt=[]
otherStockholderEquity=[]
propertyPlantEquipment=[]
totalCurrentAssets=[]
longTermInvestments=[]
netTangibleAssets=[]
netReceivables=[]
longTermDebt=[]
inventory=[]
accountsPayable=[]

In [15]:
# Parse data and add it to lists

#Function with name of Column
def collect_b(column):
     #Create local variable from the column's name
    b=globals()[column]
     #Marker for NaN value   
    NA_filler=0 
    #Number of sub-tables
    for list_number in range(4):
        #Pick particular column from sub-table
        for zz in balance_sheet.values[0][list_number]:
            #Take value from the sub-table & column
            if zz==column:
                if "fmt" not in balance_sheet.values[0][list_number][column]:xx="NaN"
                else: xx=balance_sheet.values[0][list_number][column]['fmt']
                
                #If value exist it disable NaN value checker
                NA_filler=1
                #Remove "k" letters and convert number in Mio and add to list
                if xx == None:
                    xx="NaN"
                    b.append(xx)
                elif 'k' in xx:
                    ff=xx.strip('k')
                    ff=float(ff)/1000
                    b.append(ff)
                #Remove "M" letter and add to list
                elif  'M' in xx:
                    tt=xx.strip('M')
                    b.append(tt)
                elif  'B' in xx:
                    mm=xx.strip('B')
                    mm=float(mm)*1000
                    b.append(mm)
                #Add to list if all conditions full-field
                else: 
                    b.append(xx)
        #Add NaN value if value is not exsist 
        if NA_filler==0:
            b.append("NaN")

In [16]:
for column in sectors_b:
    collect_b(column)

In [17]:
income_statement=data['incomeStatementHistory.incomeStatementHistory']

In [18]:
# Check number of lists
print("Number of sub-tables:", len(income_statement[0]))
print("Number of columns in 1st sub-table:", len(income_statement[0][0]))
print("Number of columns in 2nd sub-table:", len(income_statement[0][1]))
print("Number of columns in 3rd sub-table:", len(income_statement[0][2]))
print("Number of columns in 4th sub-table:", len(income_statement[0][3]))

Number of sub-tables: 4
Number of columns in 1st sub-table: 24
Number of columns in 2nd sub-table: 24
Number of columns in 3rd sub-table: 24
Number of columns in 4th sub-table: 24


In [19]:
sectors_l=[]
len(income_statement[0][2])
for z in income_statement[0][2]:
    sectors_l.append(z)
sectors_l.remove('maxAge')
print(sectors_l)

['researchDevelopment', 'effectOfAccountingCharges', 'incomeBeforeTax', 'minorityInterest', 'netIncome', 'sellingGeneralAdministrative', 'grossProfit', 'ebit', 'endDate', 'operatingIncome', 'otherOperatingExpenses', 'interestExpense', 'extraordinaryItems', 'nonRecurring', 'otherItems', 'incomeTaxExpense', 'totalRevenue', 'totalOperatingExpenses', 'costOfRevenue', 'totalOtherIncomeExpenseNet', 'discontinuedOperations', 'netIncomeFromContinuingOps', 'netIncomeApplicableToCommonShares']


In [20]:
researchDevelopment=[]
effectOfAccountingCharges=[]
incomeBeforeTax=[]
minorityInterest=[]
netIncome=[]
sellingGeneralAdministrative=[]
grossProfit=[]
ebit=[]
operatingIncome=[]
otherOperatingExpenses=[]
interestExpense=[]
extraordinaryItems=[]
nonRecurring=[]
otherItems=[]
incomeTaxExpense=[]
totalRevenue=[]
totalOperatingExpenses=[]
costOfRevenue=[]
totalOtherIncomeExpenseNet=[]
discontinuedOperations=[]
netIncomeFromContinuingOps=[]
netIncomeApplicableToCommonShares=[]

In [21]:
# Parse data and add it to lists

#Function with name of Column
def collect_l(column):
     #Create local variable from the column's name
    b=globals()[column]
     #Marker for NaN value   
    NA_filler=0 
    #Number of sub-tables
    for list_number in range(4):
        #Pick particular column from sub-table
        for zz in income_statement.values[0][list_number]:
            #Take value from the sub-table & column
            if zz==column:
                if "fmt" not in income_statement.values[0][list_number][column]:xx="NaN"
                else: xx=income_statement.values[0][list_number][column]['fmt']
                #If value exist it disable NaN value checker
                NA_filler=1
                #Remove "k" letters and convert number in Mio and add to list
                if xx == None:
                    xx="NaN"
                    b.append(xx)
                elif 'k' in xx:
                    ff=xx.strip('k')
                    ff=float(ff)/1000
                    b.append(ff)
                #Remove "M" letter and add to list
                elif  'M' in xx:
                    tt=xx.strip('M')
                    b.append(tt)
                elif  'B' in xx:
                    mm=xx.strip('B')
                    mm=float(mm)*1000
                    b.append(mm)
                #Add to list if all conditions full-field
                else: 
                    b.append(xx)
        #Add NaN value if value is not exsist 
        if NA_filler==0:
            b.append("NaN")

In [22]:
for column in sectors_l:
    collect_l(column)

In [23]:
# Create collected DataFrame
Cash_flow_data = {
'Investments':investments,
"ChangeToLiabilities":changeToLiabilities,
"CashflowsFromInvestingActivities":totalCashflowsFromInvestingActivities,
"NetBorrowings":netBorrowings,
"CashFromFinancingActivities":totalCashFromFinancingActivities,
"ChangeToOperatingActivities":changeToOperatingActivities,
"IssuanceOfStock":issuanceOfStock,
"NetIncome":netIncome,
"ChangeInCash":changeInCash,
"Date":endDate,
"RepurchaseOfStock":repurchaseOfStock,
"CashFromOperatingActivities":totalCashFromOperatingActivities,
"Depreciation":depreciation,
"ChangeToInventory":changeToInventory,
"ChangeToAccountReceivables":changeToAccountReceivables,
"OtherCashflowsFromFinancingActivities":otherCashflowsFromFinancingActivities,
"ChangeToNetincome":changeToNetincome,
"CapitalExpenditures":capitalExpenditures,
"DividendsPaid":dividendsPaid,
"OtherCashflowsFromInvestingActivities":otherCashflowsFromInvestingActivities,
"EffectOfExchangeRate":effectOfExchangeRate,
"Inventory":inventory,
"B_IntangibleAssets":intangibleAssets,
"B_CapitalSurplus":capitalSurplus,
"B_TotalLiab":totalLiab,
"B_TotalStockholderEquity":totalStockholderEquity,
"B_DeferredLongTermLiab":deferredLongTermLiab,
"B_OtherCurrentLiab":otherCurrentLiab,
"B_TotalAssets":totalAssets,
"B_CommonStock":commonStock,
"B_OtherCurrentAssets":otherCurrentAssets,
"B_RetainedEarnings":retainedEarnings,
"B_OtherLiab":otherLiab,
"B_GoodWill":goodWill,
"B_TreasuryStock":treasuryStock,
"B_OtherAssets":otherAssets,
"B_Cash":cash,
"B_TotalCurrentLiabilities":totalCurrentLiabilities,
"B_DeferredLongTermAssetCharges":deferredLongTermAssetCharges,
"B_ShortLongTermDebt":shortLongTermDebt,
"B_OtherStockholderEquity":otherStockholderEquity,
"B_PropertyPlantEquipment":propertyPlantEquipment,
"B_TotalCurrentAssets":totalCurrentAssets,
"B_NetTangibleAssets":netTangibleAssets,
"B_NetReceivables":netReceivables,
"B_LongTermDebt":longTermDebt,
"B_AccountsPayable":accountsPayable,
"I_researchDevelopment":researchDevelopment,
"I_EffectOfAccountingCharges":effectOfAccountingCharges,
"I_IncomeBeforeTax":incomeBeforeTax,
"I_MinorityInterest":minorityInterest,
"I_NetIncome":netIncome,
"I_SellingGeneralAdministrative":sellingGeneralAdministrative,
"I_GrossProfit":grossProfit,
"I_Ebit":ebit,
"I_OperatingIncome":operatingIncome,
"I_OtherOperatingExpenses":otherOperatingExpenses,
"I_InterestExpense":interestExpense,
"I_ExtraordinaryItems":extraordinaryItems,
"I_NonRecurring":nonRecurring,
"I_OtherItems":otherItems,
"I_IncomeTaxExpense":incomeTaxExpense,
"I_TotalRevenue":totalRevenue,
"I_TotalOperatingExpenses":totalOperatingExpenses,
"I_CostOfRevenue":costOfRevenue,
"I_TotalOtherIncomeExpenseNet":totalOtherIncomeExpenseNet,
"I_DiscontinuedOperations":discontinuedOperations,
"I_NetIncomeFromContinuingOps":netIncomeFromContinuingOps,
"I_NetIncomeApplicableToCommonShares ":netIncomeApplicableToCommonShares 
}

In [32]:
# Check a result after parsing
finish=0
for x in Cash_flow_data.values():
    finish=finish+1
    print(finish,x,len(x))

1 ['NaN', 'NaN', 'NaN', 'NaN'] 4
2 ['53.7', '-165.5', '-208.9', '175.13'] 4
3 ['-75.9', '-89.2', 2080.0, '119.2'] 4
4 ['-193.6', '-179', '-225.5', '527.27'] 4
5 ['-301.5', '-198.2', -3010.0, '-428.73'] 4
6 ['14.1', '8.8', '-14.8', '-94'] 4
7 [0.4, '2.6', '3.1', '3.87'] 4
8 ['-424.4', -2970.0, 1470.0, '522.67'] 4
9 ['-178.8', '381.5', '-265.2', '313.27'] 4
10 ['2021-10-31', '2020-10-31', '2019-10-31', '2018-10-31'] 4
11 ['-27.2', '-27.2', -2340.0, '-447.8'] 4
12 ['200.6', '678.2', '661.8', '614.53'] 4
13 ['749.2', '532.8', '783', '687.93'] 4
14 ['NaN', 0.1, 0.1, 0.06667000000000001] 4
15 ['-195.2', '251.6', '156.5', '-272.53'] 4
16 ['NaN', '-21.8', '-1', '-150.6'] 4
17 ['-345', 2700.0, -1570.0, '-445'] 4
18 ['-17.7', '-26.3', '-56.3', '-26.73'] 4
19 ['-81.1', '-81.1', '-439.2', '-361.47'] 4
20 ['1.7', '2.4', 2240.0, '5.67'] 4
21 ['-2', '-9.3', '-1.8', '10.2'] 4
22 ['NaN', 'NaN', 0.1, 0.2] 4
23 [4330.0, 5380.0, 5940.0, 6630.0] 4
24 ['46.8', '46.5', '44', '41'] 4
25 [7530.0, 7930.0, 8020.

In [28]:
x=0
while x<4:
    deferredLongTermAssetCharges.append("NaN")
    x=x+1

In [29]:
shortLongTermDebt.append("NaN")

In [ ]:
endDate.pop()

In [31]:
x=0
while x<8:
    endDate.pop()
    x=x+1

In [25]:
start=0
for g in Cash_flow_data.keys():
    start=start+1
    print(start, g)

1 Investments
2 ChangeToLiabilities
3 CashflowsFromInvestingActivities
4 NetBorrowings
5 CashFromFinancingActivities
6 ChangeToOperatingActivities
7 IssuanceOfStock
8 NetIncome
9 ChangeInCash
10 Date
11 RepurchaseOfStock
12 CashFromOperatingActivities
13 Depreciation
14 ChangeToInventory
15 ChangeToAccountReceivables
16 OtherCashflowsFromFinancingActivities
17 ChangeToNetincome
18 CapitalExpenditures
19 DividendsPaid
20 OtherCashflowsFromInvestingActivities
21 EffectOfExchangeRate
22 Inventory
23 B_IntangibleAssets
24 B_CapitalSurplus
25 B_TotalLiab
26 B_TotalStockholderEquity
27 B_DeferredLongTermLiab
28 B_OtherCurrentLiab
29 B_TotalAssets
30 B_CommonStock
31 B_OtherCurrentAssets
32 B_RetainedEarnings
33 B_OtherLiab
34 B_GoodWill
35 B_TreasuryStock
36 B_OtherAssets
37 B_Cash
38 B_TotalCurrentLiabilities
39 B_DeferredLongTermAssetCharges
40 B_ShortLongTermDebt
41 B_OtherStockholderEquity
42 B_PropertyPlantEquipment
43 B_TotalCurrentAssets
44 B_NetTangibleAssets
45 B_NetReceivables
46 B

In [33]:
# Rename DataFrame
df2=pd.DataFrame(Cash_flow_data)

In [34]:
# Check DataFrame
df2.head()

,Investments,ChangeToLiabilities,CashflowsFromInvestingActivities,NetBorrowings,CashFromFinancingActivities,ChangeToOperatingActivities,IssuanceOfStock,NetIncome,ChangeInCash,Date,RepurchaseOfStock,CashFromOperatingActivities,Depreciation,ChangeToInventory,ChangeToAccountReceivables,OtherCashflowsFromFinancingActivities,ChangeToNetincome,CapitalExpenditures,DividendsPaid,OtherCashflowsFromInvestingActivities,EffectOfExchangeRate,Inventory,B_IntangibleAssets,B_CapitalSurplus,B_TotalLiab,B_TotalStockholderEquity,B_DeferredLongTermLiab,B_OtherCurrentLiab,B_TotalAssets,B_CommonStock,B_OtherCurrentAssets,B_RetainedEarnings,B_OtherLiab,B_GoodWill,B_TreasuryStock,B_OtherAssets,B_Cash,B_TotalCurrentLiabilities,B_DeferredLongTermAssetCharges,B_ShortLongTermDebt,B_OtherStockholderEquity,B_PropertyPlantEquipment,B_TotalCurrentAssets,B_NetTangibleAssets,B_NetReceivables,B_LongTermDebt,B_AccountsPayable,I_researchDevelopment,I_EffectOfAccountingCharges,I_IncomeBeforeTax,I_MinorityInterest,I_NetIncome,I_SellingGeneralAdministrative,I_GrossProfit,I_Ebit,I_OperatingIncome,I_OtherOperatingExpenses,I_InterestExpense,I_ExtraordinaryItems,I_NonRecurring,I_OtherItems,I_IncomeTaxExpense,I_TotalRevenue,I_TotalOperatingExpenses,I_CostOfRevenue,I_TotalOtherIncomeExpenseNet,I_DiscontinuedOperations,I_NetIncomeFromContinuingOps,I_NetIncomeApplicableToCommonShares
0,NaN,53.7,-75.9,-193.6,-301.5,14.1,0.4,-424.4,-178.8,2021-10-31,-27.2,200.6,749.2,NaN,-195.2,NaN,-345,-17.7,-81.1,1.7,-2,NaN,4330.0,46.8,7530.0,2820.0,NaN,1220.0,10350.0,47.4,403.3,-1120.0,1020.0,3730.0,3850.0,154.2,558.4,1860.0,NaN,60,3850.0,228.6,1910.0,-5240.0,905.3,4520.0,80.9,522.2,NaN,-517.8,NaN,-424.4,1620.0,2130.0,-19.8,-19.8,NaN,-248.9,NaN,NaN,NaN,-82.7,2900.0,2920.0,773.7,-498,10.7,-435.1,-424.4
1,NaN,-165.5,-89.2,-179,-198.2,8.8,2.6,-2970.0,381.5,2020-10-31,-27.2,678.2,532.8,0.1,251.6,-21.8,2700.0,-26.3,-81.1,2.4,-9.3,NaN,5380.0,46.5,7930.0,3220.0,NaN,1210.0,11150.0,47.3,27.9,-741.3,1280.0,3840.0,3860.0,85.7,737.2,1790.0,NaN,21.4,3860.0,300.9,1540.0,-6000.0,727.6,4700.0,71.5,512.7,NaN,-2940.0,NaN,-2970.0,1440.0,2300.0,348.6,348.6,NaN,-271,NaN,NaN,NaN,34.2,3000.0,2650.0,698.7,-3290.0,5.1,-2970.0,-2970.0
2,NaN,-208.9,2080.0,-225.5,-3010.0,-14.8,3.1,1470.0,-265.2,2019-10-31,-2340.0,661.8,783,0.1,156.5,-1,-1570.0,-56.3,-439.2,2240.0,-1.8,0.1,5940.0,44,8020.0,6280.0,NaN,1210.0,14290.0,47.2,19.3,2250.0,1500.0,6670.0,3930.0,92.6,355.7,1800.0,NaN,3.7,3930.0,140.5,1450.0,-6340.0,1020.0,4710.0,105,491.7,NaN,-34.1,1.3,1470.0,1570.0,2570.0,513.8,513.8,NaN,-264.2,NaN,NaN,NaN,-16,3350.0,2830.0,777.3,-547.9,1490.0,-18.1,1470.0
3,NaN,175.13,119.2,527.27,-428.73,-94,3.87,522.67,313.27,2018-10-31,-447.8,614.53,687.93,0.06667,-272.53,-150.6,-445,-26.73,-361.47,5.67,10.2,0.2,6630.0,41,8990.0,7790.0,NaN,1800.0,16780.0,65.8,1140.0,3280.0,1680.0,6800.0,4410.0,55.5,620.9,2450.0,NaN,NaN,4410.0,144.3,3060.0,-5640.0,1240.0,4840.0,46.1,442.27,NaN,22.73,1,522.67,1290.0,2340.0,608.53,608.53,NaN,-224.4,NaN,NaN,NaN,-448.73,3170.0,2560.0,824.87,-585.8,51.27,471.47,522.67


In [35]:
# Remove NaN values
df3=df2.replace("NaN",0)

In [36]:
# Check DataFrame
df3.head()

,Investments,ChangeToLiabilities,CashflowsFromInvestingActivities,NetBorrowings,CashFromFinancingActivities,ChangeToOperatingActivities,IssuanceOfStock,NetIncome,ChangeInCash,Date,RepurchaseOfStock,CashFromOperatingActivities,Depreciation,ChangeToInventory,ChangeToAccountReceivables,OtherCashflowsFromFinancingActivities,ChangeToNetincome,CapitalExpenditures,DividendsPaid,OtherCashflowsFromInvestingActivities,EffectOfExchangeRate,Inventory,B_IntangibleAssets,B_CapitalSurplus,B_TotalLiab,B_TotalStockholderEquity,B_DeferredLongTermLiab,B_OtherCurrentLiab,B_TotalAssets,B_CommonStock,B_OtherCurrentAssets,B_RetainedEarnings,B_OtherLiab,B_GoodWill,B_TreasuryStock,B_OtherAssets,B_Cash,B_TotalCurrentLiabilities,B_DeferredLongTermAssetCharges,B_ShortLongTermDebt,B_OtherStockholderEquity,B_PropertyPlantEquipment,B_TotalCurrentAssets,B_NetTangibleAssets,B_NetReceivables,B_LongTermDebt,B_AccountsPayable,I_researchDevelopment,I_EffectOfAccountingCharges,I_IncomeBeforeTax,I_MinorityInterest,I_NetIncome,I_SellingGeneralAdministrative,I_GrossProfit,I_Ebit,I_OperatingIncome,I_OtherOperatingExpenses,I_InterestExpense,I_ExtraordinaryItems,I_NonRecurring,I_OtherItems,I_IncomeTaxExpense,I_TotalRevenue,I_TotalOperatingExpenses,I_CostOfRevenue,I_TotalOtherIncomeExpenseNet,I_DiscontinuedOperations,I_NetIncomeFromContinuingOps,I_NetIncomeApplicableToCommonShares
0,0,53.7,-75.9,-193.6,-301.5,14.1,0.4,-424.4,-178.8,2021-10-31,-27.2,200.6,749.2,0.00000,-195.2,0,-345,-17.7,-81.1,1.7,-2,0.0,4330.0,46.8,7530.0,2820.0,0,1220.0,10350.0,47.4,403.3,-1120.0,1020.0,3730.0,3850.0,154.2,558.4,1860.0,0,60,3850.0,228.6,1910.0,-5240.0,905.3,4520.0,80.9,522.2,0,-517.8,0,-424.4,1620.0,2130.0,-19.8,-19.8,0,-248.9,0,0,0,-82.7,2900.0,2920.0,773.7,-498,10.7,-435.1,-424.4
1,0,-165.5,-89.2,-179,-198.2,8.8,2.6,-2970.0,381.5,2020-10-31,-27.2,678.2,532.8,0.10000,251.6,-21.8,2700.0,-26.3,-81.1,2.4,-9.3,0.0,5380.0,46.5,7930.0,3220.0,0,1210.0,11150.0,47.3,27.9,-741.3,1280.0,3840.0,3860.0,85.7,737.2,1790.0,0,21.4,3860.0,300.9,1540.0,-6000.0,727.6,4700.0,71.5,512.7,0,-2940.0,0,-2970.0,1440.0,2300.0,348.6,348.6,0,-271,0,0,0,34.2,3000.0,2650.0,698.7,-3290.0,5.1,-2970.0,-2970.0
2,0,-208.9,2080.0,-225.5,-3010.0,-14.8,3.1,1470.0,-265.2,2019-10-31,-2340.0,661.8,783,0.10000,156.5,-1,-1570.0,-56.3,-439.2,2240.0,-1.8,0.1,5940.0,44,8020.0,6280.0,0,1210.0,14290.0,47.2,19.3,2250.0,1500.0,6670.0,3930.0,92.6,355.7,1800.0,0,3.7,3930.0,140.5,1450.0,-6340.0,1020.0,4710.0,105,491.7,0,-34.1,1.3,1470.0,1570.0,2570.0,513.8,513.8,0,-264.2,0,0,0,-16,3350.0,2830.0,777.3,-547.9,1490.0,-18.1,1470.0
3,0,175.13,119.2,527.27,-428.73,-94,3.87,522.67,313.27,2018-10-31,-447.8,614.53,687.93,0.06667,-272.53,-150.6,-445,-26.73,-361.47,5.67,10.2,0.2,6630.0,41,8990.0,7790.0,0,1800.0,16780.0,65.8,1140.0,3280.0,1680.0,6800.0,4410.0,55.5,620.9,2450.0,0,0,4410.0,144.3,3060.0,-5640.0,1240.0,4840.0,46.1,442.27,0,22.73,1,522.67,1290.0,2340.0,608.53,608.53,0,-224.4,0,0,0,-448.73,3170.0,2560.0,824.87,-585.8,51.27,471.47,522.67


In [37]:
# Save the file
df3.to_csv('dataset_QZ_v1.csv', index=False)